# Getting Started with PrimeHub Python SDK
PrimeHub Python SDK makes you automation with PrimeHub Platform.

In order to make the SDK working, you have to

- install the library with pip
- create a config file in the ~/.primehub/config.json

# Part 1: prerequisite: Configure the environment.

## 1. Install with pip
Let's install PrimeHub Python SDK with pip.

In [1]:
!pip install primehub-python-sdk

## 2. Request the API Token
In order to get the token, you have to have an account in the PrimeHub cluster, the following process will ask you loing with your account.



In [2]:
# PLEASE UPDATE PRIMEHUB_CLUSTER to your cluster
PRIMEHUB_CLUSTER = 'http://primehub-python-sdk.primehub.io'

In [3]:
import os
from primehub import PrimeHub, PrimeHubConfig

ph = PrimeHub(PrimeHubConfig())
if not os.path.isfile(os.path.join(os.getenv("HOME"), ".primehub/config.json")):
    ph.config.generate(PRIMEHUB_CLUSTER)

In [4]:
ph = PrimeHub(PrimeHubConfig())
if ph.is_ready():
    print("PrimeHub Python SDK setup successfully")
    print("Current Group:", ph.primehub_config.current_group)
else:
    print("PrimeHub Python SDK couldn't get the group information, please check the configuration.")

PrimeHub Python SDK setup successfully
Current Group: {'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a', 'name': 'phusers', 'displayName': 'primehub users'}


## 3. Check the account is Admin account

Use `ph.me.me` to know that the account is admin account.

In [5]:
account_information = ph.me.me()
account_information['isAdmin']

True

## 4. Clean up the previous testing data.

We need to clean up the previous data to prevent the notebook error,

In [6]:
import pandas as pd

In [7]:
# Clean up volume data we created before.

for volume_name in ["pv-volume", "host-path-volume", "git-volume", "env-volume"]:
    df_list = pd.DataFrame.from_records(list(ph.admin.volumes.list()))
    if len(df_list) > 0:
        target_id = list(df_list.loc[df_list['name'].str.contains(volume_name)]["id"])

        if len(target_id) == 1:
            print("clean the group data: {}".format(target_id))
            ph.admin.volumes.delete(list(target_id)[0])

clean the group data: ['pv-volume-1668656133']
clean the group data: ['host-path-volume']
clean the group data: ['git-volume']
clean the group data: ['env-volume']


In [8]:
# Clean up "test_group_from_jupyter" group data.

df_list = pd.DataFrame.from_records(list(ph.admin.groups.list()))
if len(df_list) > 0:
    target_id = list(df_list.loc[df_list['name'] == "test_group_from_jupyter"]["id"])

    if len(target_id) == 1:
        print("clean the group data: {}".format(target_id))
        ph.admin.groups.delete(list(target_id)[0])

clean the group data: ['afc2366f-e9b4-4eec-991c-e128dd3a0b89']


# Part 2: Volume-wise operation
We will test:
    
- Create a volume of Persistent Volume type
- Create a volume of HostPath type
- Create a volume of Git sync type
- Create a volume of Env type
- Disable the Global of a volume and assign the volume multiple groups

## 1. Create a volume of Persistent Volume type


In [9]:
import time

# Create a pv volume with admin role
config = {
  "name": "pv-volume-{}".format(int(time.time())),
  "displayName": "the volume created by SDK",
  "description": "It is a PV volume",
  "type": "pv",
  "global": True,
  "pvProvisioning": "auto",
  "volumeSize": 1
}
volume = ph.admin.volumes.create(config)

In [10]:
ph.admin.volumes.get(volume["id"]) 

{'id': 'pv-volume-1668656166',
 'name': 'pv-volume-1668656166',
 'displayName': 'the volume created by SDK',
 'description': 'It is a PV volume',
 'type': 'pv',
 'pvProvisioning': 'auto',
 'volumeSize': 1,
 'enableUploadServer': False,
 'uploadServerLink': None,
 'global': True,
 'groups': []}

## 2. Create a volume of HostPath type

In [11]:
# Create a host path volume with admin role
config = {
  "name": "host-path-volume",
  "description": "",
  "type": "hostPath",
  "global": True,
  "hostPath": "/opt/data"
}
volume = ph.admin.volumes.create(config)

In [12]:
ph.admin.volumes.get(volume["id"]) 

{'id': 'host-path-volume',
 'name': 'host-path-volume',
 'displayName': 'host-path-volume',
 'description': '',
 'type': 'hostPath',
 'hostPath': '/opt/data',
 'enableUploadServer': False,
 'uploadServerLink': None,
 'global': True,
 'groups': []}

## 3. Create a volume of Git sync type


In [13]:
# Create a git sync volume with admin role
config = {
  "name": "git-volume",
  "type": "git",
  "global": True,
  "url": "https://github.com/datasets/covid-19"
}
volume = ph.admin.volumes.create(config)

In [14]:
ph.admin.volumes.get(volume["id"]) 

{'id': 'git-volume',
 'name': 'git-volume',
 'displayName': 'git-volume',
 'description': '',
 'type': 'git',
 'url': 'https://github.com/datasets/covid-19',
 'secret': None,
 'global': True,
 'groups': []}

## 4. Create a volume of Env type

In [15]:
# Create a env volume with admin role
config = {
  "name": "env-volume",
  "description": "",
  "type": "env",
  "variables": {
    "ENV": "prod",
    "LUCKY_NUMBER": "7"
  }
}
volume = ph.admin.volumes.create(config)

In [16]:
ph.admin.volumes.get(volume["id"]) 

{'id': 'env-volume',
 'name': 'env-volume',
 'displayName': 'env-volume',
 'description': '',
 'type': 'env',
 'variables': {'ENV': 'prod', 'LUCKY_NUMBER': '7'},
 'global': False,
 'groups': []}

## 5. Disable the Global and assign the instance type to multiple groups

First, we use `ph.admin.groups.create` to create the group and Use `ph.admin.instancetypes.update` to:
- Disable global instance
- Assign the instance type to multiple groups

In [17]:
# Create a group with admin role
config = {
  "name": "test_group_from_jupyter",
  "displayName": "test_group",
  "enabledDeployment": False,
  "enabledSharedVolume": False,
  "quotaCpu": 0.5,
  "quotaGpu": 0,
  "admins": "",
  "users": {
      "connect": []
  }
}
data = ph.admin.groups.create(config)

In [18]:
import pandas as pd
df_list = pd.DataFrame.from_records(list(ph.admin.groups.list()))
df_list

,id,displayName,name,admins,quotaCpu,quotaGpu,quotaMemory,projectQuotaCpu,projectQuotaGpu,projectQuotaMemory,sharedVolumeCapacity
0,ccdd52b8-db16-46ce-8a06-70343c79e82a,primehub users,phusers,phadmin,NaN,NaN,None,None,None,None,1.0
1,c9aa9754-2805-40ac-8bb2-835aa17ad9f7,test_group,test_group_from_jupyter,,0.5,0.0,None,None,None,None,NaN


In [19]:
instance_connect = []
for list_id in list(df_list['id'])[0:2]:
    instance_connect.append({"id": list_id, 'writable': False})
instance_connect[0]['writable'] = True
instance_connect

[{'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a', 'writable': True},
 {'id': 'c9aa9754-2805-40ac-8bb2-835aa17ad9f7', 'writable': False}]

In [20]:
# Update the Volume
config = {
  "global": False,
  "groups": {
    "connect": instance_connect
  },
}
ph.admin.volumes.update(volume["id"], config)

{'id': 'env-volume'}

In [21]:
# Get the volume information

ph.admin.volumes.get(volume['id'])

{'id': 'env-volume',
 'name': 'env-volume',
 'displayName': 'env-volume',
 'description': '',
 'type': 'env',
 'variables': {'ENV': 'prod', 'LUCKY_NUMBER': '7'},
 'global': False,
 'groups': [{'id': 'ccdd52b8-db16-46ce-8a06-70343c79e82a',
   'name': 'phusers',
   'displayName': 'primehub users',
   'writable': False},
  {'id': 'c9aa9754-2805-40ac-8bb2-835aa17ad9f7',
   'name': 'test_group_from_jupyter',
   'displayName': 'test_group',
   'writable': False}]}